In [ ]:
PORT = 5000
env = 'LOCAL' # LOCAL | ICL

In [ ]:
from pathlib import Path

if env == 'LOCAL':
    WORK_DIR = Path('.')
elif env == 'ICL':
    import os
    WORK_DIR = Path('/vol/bitbucket/') / os.getenv("USER") / 'worker'

In [ ]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from src.constants import UI_URL, WORKER_URL
from src.models import ProcessRequest

nest_asyncio.apply()

app = FastAPI()

origins = [
    str(UI_URL),
    str(WORKER_URL),
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/process")
def analyze(request: ProcessRequest):
    print('Received socket: ', request.socket)
    return {'status': 200}

uvicorn.run(app, host="0.0.0.0", port=PORT)